In [1]:
import pickle
from collections import OrderedDict
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [2]:
from exfor_tools.distribution import AngularDistribution

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
from jitr.optical_potentials import kduq, wlh

In [4]:
import rxmc

In [5]:
import elm

In [6]:
results_dir = Path("./simple_so_no_intercept")

In [7]:
neutron = (1, 0)
proton = (1, 1)

## Read in samples

In [8]:
def draw_samples(chains, draws, burn):
    return chains[:, burn:, :].reshape(
        chains.shape[0] * (chains.shape[1] - burn), chains.shape[2]
    )[np.random.choice(chains.shape[0] * (chains.shape[1] - burn), draws), :]

## read in corpora that we used for calibration

In [9]:
with open("./nn_corpus.pkl", "rb") as f:
    nn_corpus = pickle.load(f)
with open("./pp_corpus.pkl", "rb") as f:
    pp_corpus = pickle.load(f)

In [10]:
posterior_samples = rxmc.params.array_to_list(
    draw_samples(np.load(results_dir / "all_chains.npy"), 1000, 10000), nn_corpus.params
)

## Propagate ELM prior into constraint observables

In [11]:
yth_nn = []
for i, constraint in enumerate(tqdm(nn_corpus.constraints)):
    yth_nn.append(np.vstack([constraint.model(p) for p in posterior_samples]))

100%|███████████████████████████████████████████████████████████████| 65/65 [02:20<00:00,  2.16s/it]


In [12]:
yth_pp = []
for i, constraint in enumerate(tqdm(pp_corpus.constraints)):
    yth_pp.append(np.vstack([constraint.model(p) for p in posterior_samples]))

100%|█████████████████████████████████████████████████████████████| 160/160 [05:24<00:00,  2.03s/it]


## Write posterior predictive distributions to disk

In [13]:
with open(results_dir / "nn_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [14]:
with open(results_dir / "pp_elm_prior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)